# Star Schema Data Processing Pipeline
This notebook creates a star schema with dimension tables and a fact table for Airbnb data analysis.

## 1. Setup and Data Loading
Import required libraries and load the source dataset.

In [ ]:
# Import necessary libraries
import pandas as pd
import hashlib
import os

# Load the cleaned dataset
df = pd.read_csv(os.path.join(os.path.dirname(os.path.abspath("__file__")), "..", "app", "data", "airbnb_listings_clean.csv"))

# Hash function to generate unique IDs from column values
def hash_id(*cols):
    key = "_".join([str(c) for c in cols])
    return int(hashlib.md5(key.encode()).hexdigest()[:10], 16)


C:\Users\MSI\AppData\Local\Temp\ipykernel_14040\2752770956.py:4: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(os.path.dirname(os.path.abspath("__file__")), "..", "app", "data", "airbnb_listings_clean.csv"))


## 2. Create Dimension Tables
Generate dimension tables with unique IDs using hash functions.

In [ ]:
# Dimension Table 1: Location
# Extract unique location combinations and generate location_id
dim_location = df[['city','country','latitude','longitude']].drop_duplicates()

dim_location['location_id'] = dim_location.apply(
    lambda r: hash_id(r['city'], r['country'], r['latitude'], r['longitude']),
    axis=1
)

In [ ]:
# Dimension Table 2: Host
# Extract unique host attributes and generate host_id
dim_host = df[['host_is_superhost']].drop_duplicates()

dim_host['host_id'] = dim_host.apply(
    lambda r: hash_id(r['host_is_superhost']),
    axis=1
)

In [ ]:
# Dimension Table 3: Room Type
# Extract unique room type combinations and generate room_type_id
dim_room = df[['room_type','room_shared','room_private']].drop_duplicates()

dim_room['room_type_id'] = dim_room.apply(
    lambda r: hash_id(r['room_type'], r['room_shared'], r['room_private']),
    axis=1
)

In [ ]:
# Dimension Table 4: Amenities
# Extract unique amenity combinations and generate amenity_id
amen_cols = ['wifi','kitchen','air_conditioning','parking','tv','heating']
dim_amen = df[amen_cols].drop_duplicates()

dim_amen['amenity_id'] = dim_amen.apply(
    lambda r: hash_id(*r.values),
    axis=1
)

In [ ]:
# Dimension Table 5: Day Type
# Extract unique day type combinations and generate day_id
day_cols = ['day_type','is_weekend','biz','multi']
dim_day = df[day_cols].drop_duplicates()

dim_day['day_id'] = dim_day.apply(
    lambda r: hash_id(*r.values),
    axis=1
)

## 3. Create Fact Table
Merge all dimension IDs with the original data and create the fact table.

In [ ]:
# Merge original data with all dimension tables to get foreign key IDs
fact = df.merge(dim_location, on=['city','country','latitude','longitude'])
fact = fact.merge(dim_host, on=['host_is_superhost'])
fact = fact.merge(dim_room, on=['room_type','room_shared','room_private'])
fact = fact.merge(dim_amen, on=amen_cols)
fact = fact.merge(dim_day, on=day_cols)

# Select only the relevant columns for the fact table (IDs + measures)
fact_table = fact[['location_id','host_id','room_type_id','amenity_id','day_id',
                   'realSum','person_capacity','bedrooms','beds',
                   'cleanliness_rating','guest_satisfaction_overall']]

In [ ]:
# Export the fact table to CSV
fact_table.to_csv("fact_table_output.csv", index=False)

## 4. Merge IDs Back to Original Dataset
Combine the original data with the generated dimension IDs.

In [ ]:
# Reload the original dataset and fact table with IDs
df = pd.read_csv(os.path.join(os.path.dirname(os.path.abspath("__file__")), "..", "app", "data", "airbnb_listings_clean.csv"))
ids = pd.read_csv("fact_table_output.csv")

# Merge original data with IDs based on row index
final = df.merge(ids, left_index=True, right_index=True)


C:\Users\MSI\AppData\Local\Temp\ipykernel_14040\2932696193.py:1: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(os.path.dirname(os.path.abspath("__file__")), "..", "app", "data", "airbnb_listings_clean.csv"))


In [ ]:
# Display the column names to identify duplicates
print(final.columns)

Index(['realSum_x', 'room_type', 'room_shared', 'room_private',
       'person_capacity_x', 'host_is_superhost', 'multi', 'biz',
       'cleanliness_rating_x', 'guest_satisfaction_overall_x', 'bedrooms_x',
       'dist', 'metro_dist', 'attr_index', 'attr_index_norm', 'rest_index',
       'rest_index_norm', 'longitude', 'latitude', 'city', 'country',
       'day_type', 'is_weekend', 'beds_x', 'wifi', 'kitchen',
       'air_conditioning', 'parking', 'tv', 'heating', 'host_location',
       'host_listings_count', 'location_id', 'host_id', 'room_type_id',
       'amenity_id', 'day_id', 'realSum_y', 'person_capacity_y', 'bedrooms_y',
       'beds_y', 'cleanliness_rating_y', 'guest_satisfaction_overall_y'],
      dtype='object')


## 5. Clean Up Duplicate Columns
Remove duplicate columns created during the merge and rename properly.

In [ ]:
# Remove duplicate columns (suffixed with _y from the merge)
final = final.drop(columns=[
    'realSum_y',
    'person_capacity_y',
    'bedrooms_y',
    'beds_y',
    'cleanliness_rating_y',
    'guest_satisfaction_overall_y'
])

# Rename columns with _x suffix back to their original names
final = final.rename(columns={
    'realSum_x': 'realSum',
    'person_capacity_x': 'person_capacity',
    'bedrooms_x': 'bedrooms',
    'beds_x': 'beds',
    'cleanliness_rating_x': 'cleanliness_rating',
    'guest_satisfaction_overall_x': 'guest_satisfaction_overall'
})

## 6. Export Final Dataset
Save the cleaned dataset with all dimension IDs included.

In [ ]:
# Export the final dataset with dimension IDs to CSV
final.to_csv("final_raw_with_ids.csv", index=False)
